In [61]:
#This Stack Requires:
# 1. Docker
# 2. Jupyter
# 3. Selenium
# 4. RSelenium

#Practically, I wanted to create a predictor that links the year, the content, and the 3rd party views on youtube.
#I wanted to practice selinum scraping in R, and this seemed a perfect example as this already has links


In [62]:
#Start your Selinum Instance Now
#docker run -d -p 4444:4444 --shm-size="2g" selenium/standalone-chrome:4.1.2-20220131

In [6]:
use_docker <- TRUE # Uses docker to pull and augment data, requires the Selenium Instance
use_dev <- TRUE #Regenerates all values, rather than pulling from final values
Fresh_Data <- TRUE
Fresh_Computes <- TRUE

In [5]:
library(tidyverse)

In [1]:
url.data <- "https://raw.githubusercontent.com/fivethirtyeight/superbowl-ads/main/superbowl-ads.csv"

raw <- read.csv(url(url.data), header = TRUE,) 

In [2]:
head(raw)

year,brand,superbowl_ads_dot_com_url,youtube_url,funny,show_product_quickly,patriotic,celebrity,danger,animals,use_sex
2018,Toyota,https://superbowl-ads.com/good-odds-toyota/,https://www.youtube.com/watch?v=zeBZvwYQ-hA,False,False,False,False,False,False,False
2020,Bud Light,https://superbowl-ads.com/2020-bud-light-seltzer-inside-posts-brain/,https://www.youtube.com/watch?v=nbbp0VW7z8w,True,True,False,True,True,False,False
2006,Bud Light,https://superbowl-ads.com/2006-bud-light-bear-attack/,https://www.youtube.com/watch?v=yk0MQD5YgV8,True,False,False,False,True,True,False
2018,Hynudai,https://superbowl-ads.com/hope-detector-nfl-super-bowl-lii-hyundai/,https://www.youtube.com/watch?v=lNPccrGk77A,False,True,False,False,False,False,False
2003,Bud Light,https://superbowl-ads.com/2003-bud-light-hermit-crab/,https://www.youtube.com/watch?v=ovQYgnXHooY,True,True,False,False,True,True,True
2020,Toyota,https://superbowl-ads.com/2020-toyota-go-places-with-cobie-smulders/,https://www.youtube.com/watch?v=f34Ji70u3nk,True,True,False,True,True,True,False


In [ ]:
#This is the raw data from fivethirtyeight
#Now we're going to augment it

In [7]:
if (Fresh_Computes && Fresh_Data){
    remDr <- RSelenium::remoteDriver(remoteServerAddr = "localhost",
                                 port = 4444,
                                 browserName = "chrome")
    remDr$open()
}

[1] "Connecting to remote server"
$acceptInsecureCerts
[1] FALSE

$browserName
[1] "chrome"

$browserVersion
[1] "89.0.4389.90"

$chrome
$chrome$chromedriverVersion
[1] "89.0.4389.23 (61b08ee2c50024bab004e48d2b1b083cdbdac579-refs/branch-heads/4389@{#294})"

$chrome$userDataDir
[1] "/tmp/.com.google.Chrome.b0rIGu"


$`goog:chromeOptions`
$`goog:chromeOptions`$debuggerAddress
[1] "localhost:43973"


$networkConnectionEnabled
[1] FALSE

$pageLoadStrategy
[1] "normal"

$platformName
[1] "linux"

$proxy
named list()

$`se:cdp`
[1] "http://172.17.0.2:4444/session/606bdf3cd2071de5273a88c83ec74119/se/cdp"

$setWindowRect
[1] TRUE

$strictFileInteractability
[1] FALSE

$timeouts
$timeouts$implicit
[1] 0

$timeouts$pageLoad
[1] 300000

$timeouts$script
[1] 30000


$unhandledPromptBehavior
[1] "dismiss and notify"

$`webauthn:extension:largeBlob`
[1] TRUE

$`webauthn:virtualAuthenticators`
[1] TRUE

$id
[1] "606bdf3cd2071de5273a88c83ec74119"



In [140]:
print(raw[3,]['brand'])


      brand
3 Bud Light


In [9]:
if (Fresh_Computes && Fresh_Data){
    remDr$navigate("https://www.youtube.com/watch?v=zeBZvwYQ-hA") #Entering our URL gets the browser to navigate to the page
    remDr$screenshot(display = TRUE)
}

In [12]:
if (Fresh_Computes && Fresh_Data){
    webElem <- remDr$findElement(using = "css", "[class='view-count style-scope ytd-video-view-count-renderer']")
    webElem$getElementText()
}

[[1]]
[1] "185,284 views"

In [15]:
for (row in 1:nrow(raw)) {
    print(raw[row,]['youtube_url'])
}
    

                                  youtube_url
1 https://www.youtube.com/watch?v=zeBZvwYQ-hA
                                  youtube_url
2 https://www.youtube.com/watch?v=nbbp0VW7z8w
                                  youtube_url
3 https://www.youtube.com/watch?v=yk0MQD5YgV8
                                  youtube_url
4 https://www.youtube.com/watch?v=lNPccrGk77A
                                  youtube_url
5 https://www.youtube.com/watch?v=ovQYgnXHooY
                                  youtube_url
6 https://www.youtube.com/watch?v=f34Ji70u3nk
                                  youtube_url
7 https://www.youtube.com/watch?v=-gAZRN3SCBw
                                  youtube_url
8 https://www.youtube.com/watch?v=lMs79UXam9A
                                  youtube_url
9 https://www.youtube.com/watch?v=WBvkmWDjsYc
                                   youtube_url
10 https://www.youtube.com/watch?v=J0xugdotpp8
                                   youtube_url
11 https://www.youtube.com/watc

In [21]:
c_dat =  select(raw, -youtube_url, -superbowl_ads_dot_com_url)

In [22]:
head(c_dat)

year,brand,funny,show_product_quickly,patriotic,celebrity,danger,animals,use_sex
2018,Toyota,False,False,False,False,False,False,False
2020,Bud Light,True,True,False,True,True,False,False
2006,Bud Light,True,False,False,False,True,True,False
2018,Hynudai,False,True,False,False,False,False,False
2003,Bud Light,True,True,False,False,True,True,True
2020,Toyota,True,True,False,True,True,True,False


In [24]:
unique(c_dat['brand'])

,brand
1,Toyota
2,Bud Light
4,Hynudai
7,Coca-Cola
8,Kia
10,Budweiser
15,NFL
18,Pepsi
20,Doritos
34,E-Trade


In [26]:
install.packages("RSelenium")

also installing the dependencies 'bitops', 'semver', 'XML', 'caTools', 'wdman', 'binman'



package 'bitops' successfully unpacked and MD5 sums checked
package 'semver' successfully unpacked and MD5 sums checked
package 'XML' successfully unpacked and MD5 sums checked
package 'caTools' successfully unpacked and MD5 sums checked
package 'wdman' successfully unpacked and MD5 sums checked
package 'binman' successfully unpacked and MD5 sums checked
package 'RSelenium' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Aman\AppData\Local\Temp\RtmpOi363W\downloaded_packages


In [29]:
library('RSelenium')
